In [62]:
import pandas as pd
import numpy as np
import torch

data_frame = pd.read_csv("data/train_preprocessed_outlier_tnoNorm.csv")
target_columns = data_frame.columns[164:]

# print rows counth of dataframe
print(len(data_frame))
infer_min = data_frame[target_columns].min()
infer_max = data_frame[target_columns].max()
print(f"Min values: {infer_min.values}")
print(f"Max values: {infer_max.values}")

39277
Min values: [6.30849029e-01 1.33873997e+02 1.96990260e+04 3.45940007e+03
 1.36050911e+01 3.97556994e+05]
Max values: [1.37534090e+00 1.68974129e+02 1.97145678e+04 3.68232452e+03
 1.68894152e+01 4.05884021e+05]


In [63]:
# Instantiate the models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [64]:
import os
from PIL import Image
from transformers import ViTModel, ViTFeatureExtractor
from tqdm import tqdm

data = pd.read_csv('./data/test_preprocessed_qt+2.csv')

# Construct image paths
data['image_path'] = data['id'].apply(lambda x: os.path.join('./data/test_images', f"{x}.jpeg"))

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224').to(device)

# Function to extract image features
def extract_features(image_path):
    image = Image.open(image_path).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

print("Extracting image features...")

# Extract features for each image
data['image_encodes'] = [extract_features(img) for img in tqdm(data['image_path'], desc="Extracting Features")]

print("Image feature extraction completed")


c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting image features...


Extracting Features: 100%|██████████| 6391/6391 [02:26<00:00, 43.67it/s]

Image feature extraction completed


In [65]:
image_features_array = np.vstack(data['image_encodes'].values)

image_features_df = pd.DataFrame(image_features_array, index=data.index)

data = pd.concat([data, image_features_df], axis=1)

data = data.drop(columns=['id', 'image_path', 'image_encodes'])
data.columns = data.columns.astype(str)

In [66]:
from autogluon.tabular import TabularPredictor

predictor1 = TabularPredictor.load('vit-model-qt+2-0')
predictor2 = TabularPredictor.load('vit-model-qt+2-1')
predictor3 = TabularPredictor.load('vit-model-qt+2-2')
predictor4 = TabularPredictor.load('vit-model-qt+2-3')
predictor5 = TabularPredictor.load('vit-model-qt+2-4')
predictor6 = TabularPredictor.load('vit-model-qt+2-5')


In [67]:
infer_max = torch.tensor(infer_max.to_numpy()).to(device)
infer_min = torch.tensor(infer_min.to_numpy()).to(device)

In [70]:
feature1 = predictor1.predict(data)
feature2 = predictor2.predict(data)
feature3 = predictor3.predict(data)
feature4 = predictor4.predict(data)
feature5 = predictor5.predict(data)
feature6 = predictor6.predict(data)

predictions = torch.tensor([feature1, feature2, feature3, feature4, feature5, feature6]).to(device)
predictions = torch.transpose(predictions, 0, 1).to(device)
print(predictions.shape)

predictions = predictions * (infer_max - infer_min) + infer_min

# Load the test data to get the 'id' column
data_id = pd.read_csv('./data/test.csv')

# Prepare the submission DataFrame with new predictions
submission = pd.DataFrame(predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
submission['id'] = data_id['id']
submission = submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]

# Save the modified DataFrame to a new CSV file
submission.to_csv('j228ye_vit-predict.csv', index=False)

torch.Size([6391, 6])
